In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.stats import linregress

# Fetch S&P 500 data for the last 20 years
end_date = pd.Timestamp.now()
start_date = end_date - pd.DateOffset(years=20)
sp500 = yf.download('^GSPC', start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))

# Calculate daily returns
sp500['returns'] = sp500['Adj Close'].pct_change()

# Calculate gains and losses
sp500['gain'] = np.where(sp500['returns'] > 0, sp500['returns'], 0)
sp500['loss'] = np.where(sp500['returns'] < 0, abs(sp500['returns']), 0)

# Calculate average gain and loss
window = 14
sp500['avg_gain'] = sp500['gain'].rolling(window=window, min_periods=1).mean()
sp500['avg_loss'] = sp500['loss'].rolling(window=window, min_periods=1).mean()

# Calculate RS and RSI
sp500['RS'] = sp500['avg_gain'] / sp500['avg_loss']
sp500['RSI'] = 100 - (100 / (1 + sp500['RS']))

# Initialize a DataFrame to store RSI Slope values
rsi_slope_df = pd.DataFrame(index=sp500.index)
rsi_slope_df['RSI_Slope'] = np.nan  # Initialize the column with NaN

N = 5  # Calculate the slope of the RSI over the last N days
for i in range(len(sp500) - N + 1):
    sp500_last_n = sp500['RSI'].iloc[i:i+N]
    slope, intercept, r_value, p_value, std_err = linregress(range(N), sp500_last_n)
    rsi_slope_df.iloc[i+N-1, rsi_slope_df.columns.get_loc('RSI_Slope')] = slope

# Save the RSI Slope values to a CSV file
rsi_slope_df.to_csv('sp500_rsi_slope_20_years.csv')

print("RSI Slope calculation for the last 20 years saved to sp500_rsi_slope_20_years.csv")

[*********************100%%**********************]  1 of 1 completed


RSI Slope calculation for the last 20 years saved to sp500_rsi_slope_20_years.csv
